<a href="https://colab.research.google.com/github/DIYEPH/test/blob/main/sdxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDXL with Custom LoRA on T4 GPU

This notebook sets up Stable Diffusion XL (SDXL) on a T4 GPU in Google Colab with Python 3.11, downloads the base model from Hugging Face, and applies a custom LoRA model from Hugging Face or Civitai. It generates images using the configured pipeline.

**Prerequisites:**
- Hugging Face account and token for gated model access (e.g., `stabilityai/stable-diffusion-xl-base-1.0`).
- Civitai API key if downloading LoRA from Civitai.
- Ensure Colab is set to T4 GPU (Runtime > Change runtime type > T4 GPU).

**Note:** Replace placeholders (e.g., `YOUR_HF_TOKEN`, `YOUR_CIVITAI_API_KEY`) with your actual credentials.

In [ ]:
# Install dependencies for Python 3.11
!pip install torch==2.2.0 torchvision==0.17.0 --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers==0.29.2 transformers==4.44.2 accelerate==0.33.0 safetensors==0.4.5
!pip install requests

# Verify Python version
import sys
print(sys.version)

# Check GPU availability
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

## Step 1: Authenticate and Download Models
Authenticate with Hugging Face to download the SDXL base model. Optionally, provide a Civitai API key for LoRA download.

In [ ]:
from huggingface_hub import login
import os

# Log in to Hugging Face
HF_TOKEN = "YOUR_HF_TOKEN"  # Replace with your Hugging Face token
login(HF_TOKEN)

# Set Civitai API key (if downloading from Civitai)
CIVITAI_API_KEY = "YOUR_CIVITAI_API_KEY"  # Replace with your Civitai API key or set to None
os.environ["CIVITAI_API_KEY"] = CIVITAI_API_KEY if CIVITAI_API_KEY else ""

# Download SDXL base model
from diffusers import StableDiffusionXLPipeline
base_model = "stabilityai/stable-diffusion-xl-base-1.0"
pipeline = StableDiffusionXLPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

## Step 2: Download Custom LoRA
Choose to download a LoRA model from Hugging Face or Civitai. Replace the URLs/IDs with your desired LoRA model.

In [ ]:
import requests
import os

def download_hf_lora(repo_id, filename, local_dir="./lora"):
    os.makedirs(local_dir, exist_ok=True)
    local_path = os.path.join(local_dir, filename)
    url = f"https://huggingface.co/{repo_id}/resolve/main/{filename}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    with open(local_path, "wb") as f:
        f.write(response.content)
    return local_path

def download_civitai_lora(model_id, filename, local_dir="./lora"):
    os.makedirs(local_dir, exist_ok=True)
    local_path = os.path.join(local_dir, filename)
    url = f"https://civitai.com/api/download/models/{model_id}"
    headers = {"Authorization": f"Bearer {os.environ['CIVITAI_API_KEY']}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    with open(local_path, "wb") as f:
        f.write(response.content)
    return local_path

# Example: Download LoRA (choose one method)
# Hugging Face LoRA (e.g., a hypothetical LoRA model)
lora_path = download_hf_lora(
    repo_id="username/sdxl-lora-model",  # Replace with actual Hugging Face repo ID
    filename="model.safetensors"         # Replace with actual filename
)

# Civitai LoRA (uncomment to use)
# lora_path = download_civitai_lora(
#     model_id="123456",               # Replace with Civitai model ID
#     filename="lora_model.safetensors"
# )

## Step 3: Load LoRA into Pipeline
Load the custom LoRA weights into the SDXL pipeline.

In [ ]:
# Load LoRA weights
pipeline.load_lora_weights(
    lora_path,
    adapter_name="custom_lora"
)

# Enable LoRA
pipeline.set_adapters(["custom_lora"], adapter_weights=[1.0])

# Optimize for T4 GPU
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

## Step 4: Generate Images
Configure the prompt and generate images using the SDXL pipeline with LoRA.

In [ ]:
prompt = "A futuristic cityscape at sunset, cyberpunk style, highly detailed, vibrant colors"
negative_prompt = "blurry, low quality, artifacts"

images = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    guidance_scale=7.5,
    height=1024,
    width=1024,
    num_images_per_prompt=1
).images

# Save and display the image
images[0].save("output.png")
images[0]

## Notes
- **T4 GPU Optimization**: The notebook uses `float16` precision, VAE slicing, and model CPU offloading to fit within T4 GPU memory constraints (16GB VRAM).
- **LoRA Model**: Ensure the LoRA model is compatible with SDXL. Replace placeholder repo IDs or model IDs with actual values from Hugging Face or Civitai.
- **Performance**: Adjust `num_inference_steps` and `guidance_scale` for quality vs. speed trade-offs.
- **Storage**: Generated images are saved as `output.png` in the Colab environment. Download them manually if needed.